## Imports

In [9]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import pandas as pd
import seaborn as sns
import keras
from ReadDataCG import train as trainc, test as testc, val as valc
from ReadDataLocal import train as trainl, test as testl, val as vall

Error processing .DS_Store: OpenCV(4.5.4) /private/var/folders/b7/g6qfbypj0tq32j5_trjh516r0000gn/T/pip-req-build-0cz1mvi1/opencv/modules/imgproc/src/resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Error processing .DS_Store: OpenCV(4.5.4) /private/var/folders/b7/g6qfbypj0tq32j5_trjh516r0000gn/T/pip-req-build-0cz1mvi1/opencv/modules/imgproc/src/resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Error processing .DS_Store: OpenCV(4.5.4) /private/var/folders/b7/g6qfbypj0tq32j5_trjh516r0000gn/T/pip-req-build-0cz1mvi1/opencv/modules/imgproc/src/resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Error processing .DS_Store: OpenCV(4.5.4) /private/var/folders/b7/g6qfbypj0tq32j5_trjh516r0000gn/T/pip-req-build-0cz1mvi1/opencv/modules/imgproc/src/resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Path to dataset files: /Users/Gabriela/.cache/kagglehub/datasets

### Getting the data

In [10]:
train = trainl
val = vall
test = testl

### Making the GradCam Algorithm

In [11]:

# ---- 1. Make a dummy call to initialize model10
sample_input = np.expand_dims(x_test[0], axis=0)  # or any image
_ = model10.predict(sample_input)

# ---- 2. Get last conv layer
last_conv_layer = [layer for layer in model10.layers if isinstance(layer, tf.keras.layers.Conv2D)][-1]
last_conv_layer_name = last_conv_layer.name
print("Last conv layer:", last_conv_layer_name)

# ---- 3. Get input and output tensors from actual call
input_tensor = sample_input  # (1, 150, 150, 1)

# ---- 4. Build a Grad-CAM-ready model using Functional API
# This uses the already built graph (after predict)
grad_model = keras.models.Model(
    inputs=model10.layers[0].input,
    outputs=[model10.get_layer(last_conv_layer_name).output, model10.output]
)

# ---- 5. Grad-CAM heatmap function
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    with tf.GradientTape() as tape:
        conv_output, predictions = model(img_array)
        if pred_index is None:
            pred_index = 0 if predictions.shape[-1] == 1 else tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]

    grads = tape.gradient(class_channel, conv_output)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_output = conv_output[0]
    heatmap = tf.reduce_sum(conv_output * pooled_grads, axis=-1)
    heatmap = tf.maximum(heatmap, 0)
    heatmap /= tf.reduce_max(heatmap) + tf.keras.backend.epsilon()
    return heatmap.numpy()

# ---- 6. Decode predictions (optional)
def decode_binary_predictions(preds):
    predicted_classes = (preds > 0.5).astype(int)
    class_labels = ['Pneumonia', 'Normal']
    return [class_labels[pred] for pred in predicted_classes.flatten()]

# ---- 7. Display Grad-CAM overlay
def display_gradcam(img, heatmap, alpha=0.4):
    heatmap_resized = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap_colored = cv2.applyColorMap(np.uint8(255 * heatmap_resized), cv2.COLORMAP_JET)
    img_rgb = np.repeat(img, 3, axis=-1) if img.shape[-1] == 1 else img
    superimposed = cv2.addWeighted(np.uint8(img_rgb * 255), 1 - alpha, heatmap_colored, alpha, 0)

    plt.imshow(superimposed)
    plt.axis('off')
    plt.show()

# ---- 8. Prepare full dataset
X = np.concatenate((x_train, x_val, x_test), axis=0)
X = X.reshape(-1, 150, 150, 1)

# ---- 9. Generate predictions and Grad-CAMs
preds = model10.predict(X)
predicted_labels = decode_binary_predictions(preds)
print("Predicted:", predicted_labels[:5])

# ---- 10. Show Grad-CAM overlays
for i in range(5):
    img = np.expand_dims(X[i], axis=0)
    heatmap = make_gradcam_heatmap(img, grad_model, last_conv_layer_name)
    display_gradcam(X[i], heatmap)


NameError: name 'x_test' is not defined

In [ ]:
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = keras.models.Model(
        model.inputs, [model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    grads = tape.gradient(class_channel, last_conv_layer_output)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0)
    max_val = tf.reduce_max(heatmap)
    if max_val == 0:
        heatmap = tf.zeros_like(heatmap)
    else:
        heatmap /= max_val

    return heatmap.numpy()

sample_input = np.expand_dims(x_test[0], axis=0)  # or any image
_ = model10.predict(sample_input)

# Get last convolutional layer
last_conv_layer = [layer for layer in model10.layers if isinstance(layer, tf.keras.layers.Conv2D)][-1]

print(last_conv_layer.name)
# Instead of using xception.decode_predictions, which is designed for ImageNet (1000 classes)
# We'll interpret the raw predictions from our binary classifier
def decode_binary_predictions(preds):
    """Decodes predictions for a binary classification model."""
    predicted_classes = (preds > 0.5).astype(int)  # Threshold at 0.5
    class_labels = ['Pneumonia', 'Normal']  # Assuming labels are in this order
    return [class_labels[pred] for pred in predicted_classes.flatten()]

# Prepare images
X = np.concatenate((x_train, x_val, x_test), axis = 0)
Y = np.concatenate((y_train, y_val, y_test), axis = 0)

X = X.reshape(-1, 150, 150, 1)


# Our model
ours = model10

# Remove last layer's softmax (if it was applied)
# This might be unnecessary depending on your model definition
# ours.layers[-1].activation = None

# Print what the top predicted class is using the custom decoding function
preds = ours.predict(X)
predicted_labels = decode_binary_predictions(preds)
print("Predicted:", predicted_labels[:5])  # Print first 5 predictions for illustration


heatmaps = []
for i in range(len(X)):
    img = np.expand_dims(X[i], axis=0)  # (1, 150, 150, 1)
    heatmap = make_gradcam_heatmap(img, ours, last_conv_layer.name)
    heatmaps.append(heatmap)

heatmap = np.array(heatmaps)  # Combine heatmaps from all batches


In [ ]:

for i in range(5):  # Adjust how many samples you want to visualize
    heatmap = heatmaps[i]
    input_img = X[i].squeeze()  # Shape: (150, 150), normalized (0–1)

    # Resize heatmap to input size
    heatmap_resized = cv2.resize(heatmap, (150, 150))
    heatmap_uint8 = np.uint8(255 * heatmap_resized)
    heatmap_color = cv2.applyColorMap(heatmap_uint8, cv2.COLORMAP_JET)

    # Convert grayscale image to RGB
    input_rgb = cv2.cvtColor(np.uint8(input_img * 255), cv2.COLOR_GRAY2RGB)

    # Overlay heatmap
    overlay = cv2.addWeighted(input_rgb, 0.6, heatmap_color, 0.3, 0)

    # Plot original + overlay
    plt.figure(figsize=(6, 3))
    plt.suptitle(f"Prediction: {predicted_labels[i]}")

    plt.subplot(1, 2, 1)
    plt.title("Original Image")
    plt.imshow(input_rgb)
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.title("Grad-CAM Overlay")
    plt.imshow(overlay)
    plt.axis('off')

    plt.tight_layout()
    plt.show()